## 1. Importaciones y Configuración Inicial

Importamos las bibliotecas necesarias del Agent Framework, incluyendo componentes para:
- **WorkflowBuilder**: Construcción de grafos de flujo
- **AgentExecutor**: Ejecución de agentes
- **Condiciones y Casos**: Routing condicional
- **Azure OpenAI**: Cliente de chat con modelos GPT

In [ ]:
import asyncio
import os
from dataclasses import dataclass
from typing import Any, Literal
from uuid import uuid4

from typing_extensions import Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    executor,
    Case,
    Default,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

print("✅ Importaciones completadas")

## 2. Modelos de Datos con Pydantic

Definimos modelos de datos estructurados que garantizan respuestas consistentes de los agentes:

### DetectionResult
- **is_spam**: Booleano que determina el routing del workflow
- **reason**: Explicación legible de la clasificación
- **email_content**: Email original para procesamiento downstream

### EmailResponse
- **response**: Borrador de respuesta profesional generada por el asistente

In [ ]:
class DetectionResult(BaseModel):
    """Representa el resultado de la detección de spam."""
    # is_spam impulsa la decisión de routing tomada por las condiciones de edge
    is_spam: bool
    # Justificación legible por humanos del detector
    reason: str
    # El agente debe incluir el email original para que los agentes downstream operen sin recargar contenido
    email_content: str

class EmailResponse(BaseModel):
    """Representa la respuesta del asistente de email."""
    # El borrador de respuesta que un usuario podría copiar o enviar
    response: str

print("✅ Modelos de datos definidos")

## 3. Función de Condición para Routing

Esta función factory crea predicados de condición para las aristas (edges) del workflow:

### Funcionamiento:
1. Recibe un valor esperado (True/False para spam)
2. Retorna una función que evalúa si el mensaje coincide con esa expectativa
3. Parsea de forma segura el `DetectionResult` desde el JSON de respuesta
4. **Fail-safe**: En caso de error, retorna False para evitar routing incorrecto

Esta estrategia garantiza que solo los mensajes correctamente clasificados tomen cada ruta.

In [ ]:
def get_condition(expected_result: bool):
    """Crea un callable de condición que enruta basándose en DetectionResult.is_spam."""

    # La función retornada se usará como predicado de edge.
    # Recibe lo que sea que produjo el executor upstream.
    def condition(message: Any) -> bool:
        # Guardia defensiva. Si aparece un no-AgentExecutorResponse, permite pasar el edge para evitar callejones sin salida.
        if not isinstance(message, AgentExecutorResponse):
            return True

        try:
            # Preferir parsear un DetectionResult estructurado desde el texto JSON del agente.
            # Usar model_validate_json asegura seguridad de tipos y lanza si la forma es incorrecta.
            detection = DetectionResult.model_validate_json(message.agent_run_response.text)
            # Enrutar solo cuando el flag de spam coincide con la ruta esperada.
            return detection.is_spam == expected_result
        except Exception:
            # Fallar cerrado en errores de parseo para no enrutar accidentalmente a la ruta incorrecta.
            # Retornar False previene que este edge se active.
            return False

    return condition

print("✅ Función de condición creada")

## 4. Executors del Workflow

Los executors son nodos en el grafo del workflow que procesan mensajes:

### handle_email_response
- Procesa emails legítimos
- Parsea la respuesta estructurada del asistente
- Emite el output final del workflow

### handle_spam_classifier_response
- Maneja emails clasificados como spam
- Verifica que realmente sea spam (doble verificación)
- Marca el email como spam y finaliza

### to_email_assistant_request
- **Transformador** entre agentes
- Extrae el contenido del email del resultado de detección
- Crea una nueva petición para el asistente de email
- Permite que los agentes se comuniquen sin acoplamiento directo

In [ ]:
@executor(id="send_email")
async def handle_email_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Maneja emails legítimos redactando una respuesta profesional."""
    # Downstream del asistente de email. Parsear un EmailResponse validado y emitir el output del workflow.
    email_response = EmailResponse.model_validate_json(response.agent_run_response.text)
    await ctx.yield_output(f"Email enviado:\n{email_response.response}")

@executor(id="handle_spam")
async def handle_spam_classifier_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Maneja emails spam marcándolos apropiadamente."""
    # Ruta de spam. Confirmar el DetectionResult y emitir el output del workflow. Proteger contra input no-spam accidental.
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)
    if detection.is_spam:
        await ctx.yield_output(f"Email marcado como spam: {detection.reason}")
    else:
        # Esto indica que el predicado de routing y el contrato del executor están desincronizados.
        raise RuntimeError("Este executor solo debe manejar mensajes spam.")

@executor(id="to_email_assistant_request")
async def to_email_assistant_request(
    response: AgentExecutorResponse, ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    """Transforma la respuesta de detección de spam en una petición para el asistente de email."""
    # Parsear el resultado de detección y extraer el contenido del email para el asistente
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)

    # Crear una nueva petición para el asistente de email con el contenido del email original
    request = AgentExecutorRequest(
        messages=[ChatMessage(Role.USER, text=detection.email_content)],
        should_respond=True
    )
    await ctx.send_message(request)

print("✅ Executors del workflow definidos")

## 5. Construcción del Workflow y Ejecución

### Creación de Agentes

1. **Cliente de Chat**: Usa Azure OpenAI con autenticación de Azure CLI
2. **Agente de Detección de Spam**: 
   - Analiza emails e identifica spam
   - Usa `response_format` para garantizar JSON estructurado
   - Retorna `DetectionResult`

3. **Agente Asistente de Email**:
   - Redacta respuestas profesionales
   - También usa `response_format` para salida confiable
   - Retorna `EmailResponse`

### Construcción del Grafo

El `WorkflowBuilder` crea un grafo dirigido con:
- **Nodo inicial**: spam_detection_agent
- **Ruta No-Spam**: detector → transformador → asistente → enviar
- **Ruta Spam**: detector → manejador_spam
- **Condiciones**: Evalúan `is_spam` para routing

In [ ]:
async def main() -> None:
    # Crear agentes
    # AzureCliCredential usa tu login actual de az. Esto evita embeber secretos en el código.
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential(),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        deployment_name=os.getenv("MODEL"))

    # Agente 1. Clasifica spam y retorna un objeto DetectionResult.
    # response_format asegura que el LLM retorna JSON parseable para el modelo Pydantic.
    spam_detection_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "Eres un asistente de detección de spam que identifica emails spam. "
                "Siempre retorna JSON con campos is_spam (bool), reason (string), y email_content (string). "
                "Incluye el contenido original del email en email_content."
            ),
            response_format=DetectionResult,
        ),
        id="spam_detection_agent",
    )

    # Agente 2. Redacta una respuesta profesional. También usa salida JSON estructurada para confiabilidad.
    email_assistant_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "Eres un asistente de email que ayuda a usuarios a redactar respuestas profesionales a emails. "
                "Tu input podría ser un objeto JSON que incluye 'email_content'; basa tu respuesta en ese contenido. "
                "Retorna JSON con un solo campo 'response' conteniendo la respuesta redactada."
            ),
            response_format=EmailResponse,
        ),
        id="email_assistant_agent",
    )

    # Construir el grafo del workflow.
    # Comenzar en el detector de spam.
    # Si no es spam, saltar a un transformador que crea un nuevo AgentExecutorRequest,
    # luego llamar al asistente de email, luego finalizar.
    # Si es spam, ir directamente al manejador de spam y finalizar.
    workflow = (
        WorkflowBuilder()
        .set_start_executor(spam_detection_agent)
        # Ruta no-spam: transformar respuesta → request para asistente → asistente → enviar email
        .add_edge(spam_detection_agent, to_email_assistant_request, condition=get_condition(False))
        .add_edge(to_email_assistant_request, email_assistant_agent)
        .add_edge(email_assistant_agent, handle_email_response)
        # Ruta spam: enviar a manejador de spam
        .add_edge(spam_detection_agent, handle_spam_classifier_response, condition=get_condition(True))
        .build()
    )

    # Leer contenido del Email del archivo de muestra de recursos.
    # Esto mantiene el ejemplo determinístico ya que el modelo ve el mismo email en cada ejecución.
    email_path = os.path.join(os.path.dirname(os.path.dirname(os.path.realpath(__file__))), "resources", "email.txt")

    with open(email_path) as email_file:  # noqa: ASYNC230
        email = email_file.read()

    print("📧 Contenido del email a procesar:")
    print("="*60)
    print(email)
    print("="*60)
    print()

    # Ejecutar el workflow. Ya que el inicio es un AgentExecutor, pasar un AgentExecutorRequest.
    # El workflow se completa cuando queda inactivo (no hay más trabajo por hacer).
    request = AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=email)], should_respond=True)
    events = await workflow.run(request)
    outputs = events.get_outputs()
    if outputs:
        print(f"🎯 Output del Workflow: {outputs[0]}")

print("✅ Función main definida")

## 6. Ejecución del Workflow

Ejecutamos el workflow completo de forma asíncrona. El sistema:
1. Lee el email de ejemplo
2. Lo pasa al detector de spam
3. Según la clasificación, toma una de dos rutas
4. Procesa el resultado y emite el output final

In [ ]:
# Ejecutar el workflow
await main()

## 7. Análisis y Conclusiones

### Ventajas de este Patrón:

1. **Separación de Responsabilidades**: Cada agente tiene un propósito único y bien definido
2. **Routing Condicional Robusto**: Las condiciones type-safe previenen errores de enrutamiento
3. **Salida Estructurada**: `response_format` garantiza JSON válido y parseable
4. **Composición Modular**: Los agentes se pueden combinar en diferentes flujos
5. **Manejo de Errores**: Fail-safe conditions previenen rutas incorrectas

### Casos de Uso Prácticos:

- **Sistemas de Triaje de Emails**: Clasificación automática y respuesta
- **Moderación de Contenido**: Filtrado de spam en foros o redes sociales
- **Enrutamiento de Tickets**: Asignación automática basada en contenido
- **Workflows de Aprobación**: Routing basado en reglas de negocio

### Extensiones Posibles:

1. **Múltiples Categorías**: Extender más allá de spam/no-spam
2. **Machine Learning**: Integrar modelos personalizados de clasificación
3. **Persistencia**: Guardar resultados en base de datos
4. **Notificaciones**: Enviar alertas en tiempo real
5. **Análisis**: Dashboard de métricas de detección